# Generating ecoSPold files for WWT

Pascal Lesage notes for ICRA

## Description

The WWT LCI tool must generate importable ecoSpold2 tools.  
An example ecoSpold already in ecoinvent is found [here](zzz).  
This document shows: 
  - how to inform all the required fields  
  - how to generate the ecoSPold files

## Standard inputs

In [1]:
import os
import pandas as pd
import pickle # Used temporarily to access a MasterData dictionary - check if still useful at the end of the project.
from lxml import objectify #Convert XML to dict

In [2]:
import pandas.core.indexes
import sys
sys.modules['pandas.indexes'] = pandas.core.indexes

## Guillaume Bourgault (GB) code

This document relies heavily on the code prepared by GB and distributed in July (`spold2_writer_use.py`).  

Some slight modifications were made to make it easier to use with the WWT tool ==> see `spold_writer_

The GB code is meant to be executed from a .py file, and uses the `os.path.realpath(__file__)` function as a starting point to identify where files are found. Because I was running the functions from outside the script (in Notebooks or in an interpreter), I created a variable `root_dir = os.path.realpath(__file__)` in the file to avoid this problem.  

To run commands from the Notebooks, one can use the [%run](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-run) magic command

In [3]:
os.chdir(r'C:\mypy\code\wastewater_treatment_tool\waste_water_tool')
%run spold2_writer_functions.py
#%run spold2_writer_use.py

## Master data

The ecoinvent database contains master data for the following entities: Activity Names, Classifications, Companies, Compartments, Exchanges (Elementary and Intermediate), Geographies, Languages, Market Models, Parameters, Persons, Properties, Scenarios, Sources, Tags and Units. 

There are discussions underway to have the tool access the master data on the ecoinvent/IFU server. However, for now, this has not yet been resolved, and many at the ecoinvent Center do not feel this is very important because the amount of master data used for the WWT datasets is not that important, and because the master data could be stored on the server that will host the WWT tool and easily be regularly updated.

For now, I will use the master data that is downloaded on my computer via the [ecoEditor](http://www.ecoinvent.org/data-provider/data-provider-toolkit/ecoeditor/ecoeditor.html).  
Guillaume of the ecoinvent Center (henceforth GB) has written the following code to help **find the master data**:

In [4]:
master_data_folder = find_current_MD_path()
master_data_folder

'C://Users\\Pascal Lesage\\Documents\\ecoinvent\\EcoEditor\\xml\\MasterData\\Production'

Here are the **contents of the master data directory**:

In [5]:
os.listdir(master_data_folder)

['ActivityIndex.xml',
 'ActivityNames.xml',
 'Classifications.xml',
 'Companies.xml',
 'Compartments.xml',
 'Context.xml',
 'DeletedMasterData.xml',
 'ElementaryExchanges.xml',
 'ExchangeActivityIndex.xml',
 'Geographies.xml',
 'IntermediateExchanges.xml',
 'Languages.xml',
 'MacroEconomicScenarios.xml',
 'Parameters.xml',
 'Persons.xml',
 'Properties.xml',
 'Sources.xml',
 'SystemModels.xml',
 'Tags.xml',
 'UnitConversions.xml',
 'Units.xml',
 'user']

There are multiple ways of looking at this data: directly with a text editor, via Python using `lxml` or ElementTree.  
I'm not sure what you are more comfortable with. I will from now on simply paste XML as text in a text editor. Here is what that would look like for the first elementary exchange (ecoinvent term for "elementary flow", or flow of a substance from a unit process, or *activity* in ecoinvent, and the environment):

```XML
<?xml version="1.0" encoding="utf-8"?>
<validElementaryExchanges majorRelease="3" minorRelease="0" majorRevision="0" minorRevision="38930" xmlns="http://www.EcoInvent.org/EcoSpold02">
  <elementaryExchange id="38a622c6-f086-4763-a952-7c6b3b1c42ba" unitId="487df68b-4994-4027-8fdc-a4dc298257b7" casNumber="000110-63-4">
    <name xml:lang="en">1,4-Butanediol</name>
    <unitName xml:lang="en">kg</unitName>
    <compartment subcompartmentId="e8d7772c-55ca-4dd7-b605-fee5ae764578">
      <compartment xml:lang="en">air</compartment>
      <subcompartment xml:lang="en">urban air close to ground</subcompartment>
    </compartment>
    <synonym xml:lang="en">Butylene glycol</synonym>
    <property propertyId="6393c14b-db78-445d-a47b-c0cb866a1b25" amount="0" />
    <property propertyId="6d9e1462-80e3-4f10-b3f4-71febd6f1168" amount="0" />
    <property propertyId="a9358458-9724-4f03-b622-106eda248916" amount="0" />
    <property propertyId="c74c3729-e577-4081-b572-a283d2561a75" amount="0.533098393070742" />
    <property propertyId="3a0af1d6-04c3-41c6-a3da-92c4f61e0eaa" amount="1" />
    <property propertyId="67f102e2-9cb6-4d20-aa16-bf74d8a03326" amount="1" />
  </elementaryExchange>
```

GB has also written some code to **assemble all the master data in one dictionary**, **`MD`**, where:  
  - the keys of the dictionary are the names of the files above (`ActivityIndex`, `ActivityNames`, etc.)  
  - the values are the contents of the master data xml assembled as **pandas dataframes**.  

Here are some details:  

`get_current_MD(master_data_folder=None, pkl_folder=None, return_MD=False)`:   
  - Arguments:  
    - `master_data_folder` = dir of master data. If `None`, `find_current_MD_path`is used  
    - `pkl_folder` = directory of previously built master data dictionary. If `None` passed, the function will look where it expects it to be, i.e. ` os.path.join(os.path.dirname(os.path.realpath(__file__)),'pkl')`  
    - `return_MD`: if False, the function retunrs None, else it returns the MD
  - Compares the age of the existing master data dictionary MD (if it exists) with that of the actual master data to determine whether the disctionary can be used as-is or whether it needs to be created/updated.  
  - If it needs to be created, the function `build_MD` is called.
  - returns MD
  
`build_MD(md_fields_xls=None, master_data_folder=None, pickle_dump_folder=False, xls_dump_folder=False)`:
  - Called from `get_current_MD`, if needed. 
  - Arguments:  
    - `md_fields_xls`: path to the file `MasterData_fields.xlsx`, by default in `root_dir/documentation`. Default used if argument not passed.  
    - `master_data_folder` = dir of master data. If `None`, `find_current_MD_path`is used  
    - `pickle_dump_folder` = Directory where the pickled **`MD`** should be stored. If `None`, the MD pickle is not stored.
    - `xls_dump_folder` = Directory where the xls version of the master data should be stored. If `None`, the xls is not generated.

**Use in our case**: the `MD.pkl` dictionary is required later, and so needs to be generated.

In [6]:
MD = get_current_MD(return_MD=True)

In [7]:
# Names of dataframes:
[*MD.keys()]

['ActivityIndex',
 'ActivityNames',
 'Classifications',
 'Companies',
 'Compartments',
 'ElementaryExchanges',
 'Geographies',
 'IntermediateExchanges',
 'Languages',
 'MacroEconomicScenarios',
 'Parameters',
 'Persons',
 'Properties',
 'Sources',
 'SystemModels',
 'Tags',
 'UnitConversions',
 'Units',
 'ExchangeActivityIndex',
 'IntermediateExchanges prop.',
 'ElementaryExchanges prop.']

Sample DF

In [8]:
MD['Geographies']

,id,latitude,longitude,name,uNCode,uNRegionCode,uNSubregionCode
shortname,,,,,,,
AD,10033c48-7d7e-11de-9ae2-0019e336be3a,42.549,1.576,Andorra,0,0,0
AE,14f9e5d0-7d7e-11de-9ae2-0019e336be3a,23.549,54.163,United Arab Emirates,0,0,0
AF,0c608726-7d7e-11de-9ae2-0019e336be3a,33.677,65.216,Afghanistan,0,0,0
AG,09581fbc-7d7e-11de-9ae2-0019e336be3a,17.078,-61.783,Antigua and Barbuda,0,0,0
AI,0fefde28-7d7e-11de-9ae2-0019e336be3a,18.237,-63.032,Anguilla,0,0,0
AL,097c2556-7d7e-11de-9ae2-0019e336be3a,41.143,20.068,Albania,0,0,0
AM,09818690-7d7e-11de-9ae2-0019e336be3a,40.534,44.563,Armenia,0,0,0
AO,098771a4-7d7e-11de-9ae2-0019e336be3a,-12.296,17.544,Angola,0,0,0
AQ,105b361e-7d7e-11de-9ae2-0019e336be3a,-80.446,21.304,Antarctica,0,0,0


## Generating the ecoSpold file

This section documents the data that the tool must generate.  

The data/documentation/naming requirements are taken from several sources:  
- The [Data Quality Guidelines](http://www.ecoinvent.org/files/dataqualityguideline_ecoinvent_3_20130506.pdf)  
- Notes found in the [ecoEditor](http://www.ecoinvent.org/data-provider/data-provider-toolkit/ecoeditor/ecoeditor.html) itself
- A [dataset documentation](http://www.ecoinvent.org/files/dataset_documentation_ecoinvent_3.pdf) document

### Step 1: Generate an empty dataset

Until it is rendered (see below), the dataset will be a dictionary.  
Use the function `create_empty_dataset`  
Some fields which will remain constant are pre-filled.

In [9]:
dataset = create_empty_dataset()
dataset

{'ByProduct': [],
 'FromEnvironment': [],
 'FromTechnosphere': [],
 'Properties': [],
 'ReferenceProduct': [],
 'ToEnvironment': [],
 'activityDataset': 'activityDataset',
 'activityNameContextId': None,
 'classifications': [<__main__.GenericObject at 0x24db0621278>],
 'dataEntryBy': <__main__.GenericObject at 0x24db103a240>,
 'dataGeneratorAndPublication': <__main__.GenericObject at 0x24db103a0f0>,
 'energyValues': None,
 'fileAttributes': <__main__.GenericObject at 0x24db0924fd0>,
 'geographyContextId': None,
 'inheritanceDepth': 0,
 'isDataValidForEntirePeriod': 'true',
 'macroEconomicScenarioComment': None,
 'macroEconomicScenarioContextId': None,
 'macroEconomicScenarioId': 'd9f57f0a-a01f-42eb-a57b-8f18d6635801',
 'macroEconomicScenarioName': 'Business-as-Usual',
 'masterAllocationPropertyId': None,
 'masterAllocationPropertyIdOverwrittenByChild': None,
 'originalActivityDataset': None,
 'parentActivityContextId': None,
 'parentActivityId': None,
 'specialActivityType': 0,
 'tag':

### ActivityIndex
Collection of many things that need to be generated from user input.  
There are a series of functions to run (see below), and then the final function to put it all together is `generate_activityIndex(dataset)`

#### (1) Activity name

From ecoEditor:
>Activity Name  
>The name describes the activity that is represented by this dataset. The activity name can only be edited when a new dataset is created. If you want to use this dataset under a new activity name, you need to create a new dataset with the desired name, using the current dataset as a template (menu File..., New..., FromExistingDataset).  
>Length: 120  
>Required: Yes  
>EcoSpold02 FieldId: 100  

From DQG:
>Activity names are spelled with lower case starting letter, i.e. “lime production”, not “Lime production”. 

In the case of the WWT datasets, the name will depend on the situation: 

- CASE 1: treatment of the wastewater from a specific source in an "average" WWTP  
- CASE 2: treatment of the wastewater from a specific source in a "specific" WWTP technology/capacity  
- CASE 3: treatment of average wastewater in an "average" WWTP  
- CASE 4: treatment of average wastewater in a "specific" WWTP technology/capacity  

I have written a function `create_WWT_activity_name` that generates a valid name based on three arguments that the tool will need to get from the user:  
- `WW_type` = two choices only: average, or "from x" (e.g. "from steel production", "from residence")
- `technology`: TBD  
- `capacity` = two choices only: 'average' or int representing the yearly capacity in l/year. A check on type should be done.

This function is later used by a second function, `generate_WWT_activity_name`, see below.

In [10]:
def create_WWT_activity_name(WW_type, technology, capacity):
    if WW_type == 'average':
        WW_type_str = ", average"
    else:
        WW_type_str = " {}".format(WW_type)
    
    if technology == 'average':
        technology_str = ""
    else:
        technology_str = "{}, ".format(technology)
    
    if capacity == 'average':
        capacity_str = "average capacity"
    else:
        capacity_str = "capacity {:.1E}l/year".format(capacity).replace('+', '').replace('E0', 'E').replace('.0', '')
    
    return "treatment of wastewater{}, {}{}".format(WW_type_str, technology_str, capacity_str)

Tests (by eye):

In [11]:
print(create_WWT_activity_name("average", "technology A", 1e9))
print(create_WWT_activity_name("from steel production", "technology A", 1e9))
print(create_WWT_activity_name("average", "average", 1e9))
print(create_WWT_activity_name("from steel production", "average", 1.1e9))
print(create_WWT_activity_name("average", "average", "average"))
print(create_WWT_activity_name("from steel production", "average", "average"))

treatment of wastewater, average, technology A, capacity 1E9l/year
treatment of wastewater from steel production, technology A, capacity 1E9l/year
treatment of wastewater, average, capacity 1E9l/year
treatment of wastewater from steel production, capacity 1.1E9l/year
treatment of wastewater, average, average capacity
treatment of wastewater from steel production, average capacity


Tests (by checking existing datasets to make sure they are consistent)

In [12]:
existing_names = list(MD['ActivityNames'].index)

In [13]:
[create_WWT_activity_name("from black chrome coating", "average", 1.1e10) in existing_names,
create_WWT_activity_name("from lorry production", "average", 4.7e10) in existing_names,
create_WWT_activity_name("average", "average", 1.6e8) in existing_names]

[True, True, True]

`generate_WWT_activity_name` has four arguments:  
- `dataset`: the dataset (dictionary) that the name should be added to  
- Same three arguments as the `create_WWT_activity_name`  

It adds the name to the dataset.  
It also adds the WW_type to the dataset (because it is needed later in the creation of the reference flow name).

In [14]:
generate_WWT_activity_name(dataset, 'from ceramic production', 'average', 5e9)

In [15]:
dataset['WW_type']

'from ceramic production'

#### (2) ActivityNameID
Use `ActivityNameID` if the activity name already exists, else generate a new `ActivityNameID`.

To check if the Master data already exists, we:  
   (1) Get the `ActivityIndex` dataframe from `MD`   
   (2) Make the name the index of the dataframe  
   (3) Check if our name is in the dataframe.  

If the name does not exist:  
   (1) Generate a UUID  
   (2) Add a "generic object" to the dataset.
   
The code for adding a generic object is simple:
```Python
    field = 'ActivityNames'
    d = {'id': dataset['activityNameId'], 
         'name': dataset['activityName']}
    dataset[field] = [GenericObject(d, field)]
```
where
```Python
class GenericObject:
    def __init__(self, d, object_type):
        self.template_name = '%s_2.xml' % object_type
        assert type(d) == dict
        for key, value in d.items():
            setattr(self, key, value)
```

How exactly these work is not clear to me, but it is not necessary to move on...  

I created the function `generate_activityNameId` that takes as argument the `dataset` and the `MD` dictionary:

In [16]:
generate_activityNameId(dataset, MD)

#### (3) Geography  
The geography chosen should correspond to a geography already in the master data. It would probably therefore be a good idea to have a drop-down list to choose the geography, and a message saying that the data supplier should communicate with ecoinvent if the geography they want is not in the master data (email address: data@ecoinvent.org).  
I have written a function `generate_geography` that takes as argument the dataset and the geography **shortname**.  

In [17]:
MD['Geographies']

,id,latitude,longitude,name,uNCode,uNRegionCode,uNSubregionCode
shortname,,,,,,,
AD,10033c48-7d7e-11de-9ae2-0019e336be3a,42.549,1.576,Andorra,0,0,0
AE,14f9e5d0-7d7e-11de-9ae2-0019e336be3a,23.549,54.163,United Arab Emirates,0,0,0
AF,0c608726-7d7e-11de-9ae2-0019e336be3a,33.677,65.216,Afghanistan,0,0,0
AG,09581fbc-7d7e-11de-9ae2-0019e336be3a,17.078,-61.783,Antigua and Barbuda,0,0,0
AI,0fefde28-7d7e-11de-9ae2-0019e336be3a,18.237,-63.032,Anguilla,0,0,0
AL,097c2556-7d7e-11de-9ae2-0019e336be3a,41.143,20.068,Albania,0,0,0
AM,09818690-7d7e-11de-9ae2-0019e336be3a,40.534,44.563,Armenia,0,0,0
AO,098771a4-7d7e-11de-9ae2-0019e336be3a,-12.296,17.544,Angola,0,0,0
AQ,105b361e-7d7e-11de-9ae2-0019e336be3a,-80.446,21.304,Antarctica,0,0,0


In [18]:
generate_geography(dataset, MD, 'GLO')

#### (4) TimePeriod (Start and end date)
Period for which the dataset is meant to be valid.  Supplied by user. It would be useful to supply default values.    

Format: 'YYYY-MM-DD'. No validation in formula now.

I have written a function `generate_time_period` that takes as argument the dataset, a start date and an end date. 

Format: 'YYYY-MM-DD'. No validation in formula now.

In [19]:
generate_time_period(dataset, start='1995-01-31', end='2020-12-31')

#### (5) Dataset ID
The dataset UUID is generated from the dataset['activityName'], dataset['geography'], dataset['startDate'], dataset['endDate']

In [20]:
generate_dataset_id(dataset)

#### Putting it all together

In [21]:
generate_activityIndex(dataset)

In [22]:
dataset

{'ActivityIndex': [<__main__.GenericObject at 0x24db0c567b8>],
 'ByProduct': [],
 'FromEnvironment': [],
 'FromTechnosphere': [],
 'Properties': [],
 'ReferenceProduct': [],
 'ToEnvironment': [],
 'WW_type': 'from ceramic production',
 'activityDataset': 'activityDataset',
 'activityName': 'treatment of wastewater from ceramic production, capacity 5E9l/year',
 'activityNameContextId': None,
 'activityNameId': '1ec0e569-0949-4b57-8ccd-ebb7ffef9811',
 'classifications': [<__main__.GenericObject at 0x24db0621278>],
 'dataEntryBy': <__main__.GenericObject at 0x24db103a240>,
 'dataGeneratorAndPublication': <__main__.GenericObject at 0x24db103a0f0>,
 'endDate': '2020-12-31',
 'energyValues': None,
 'fileAttributes': <__main__.GenericObject at 0x24db0924fd0>,
 'geography': 'GLO',
 'geographyContextId': None,
 'geographyId': '34dbbff8-88ce-11de-ad60-0019e336be3a',
 'id': '99948add-39df-50bd-8b74-f478abd63712',
 'inheritanceDepth': 0,
 'isDataValidForEntirePeriod': 'true',
 'macroEconomicScenar

### Activity description

#### includedActivitiesStart, includedActivitiesEnd

Two text fields. Describes the boundaries of the unit process.

##### Included Activities Start
Suggested text for industrial wastewater:

In [23]:
source = 'from ceramic production'
includedActivitiesStartText = "From the discharge of wastewater {} to the sewer grid.".format(source)
includedActivitiesStartText

'From the discharge of wastewater from ceramic production to the sewer system.'

Suggested text for municipal wastewater.

In [24]:
includedActivitiesStartText = "From the discharge of municipal wastewater to the sewer grid."

##### Included Activities End  
Based on the [dataset documentation](http://www.ecoinvent.org/files/dataset_documentation_ecoinvent_3.pdf) document, this section has three parts:  
>(i) what is the last activity covered resp. what is the point
of delivery of this dataset?
(ii) what activities are included (and not obvious from the name of the
activity)
(iii) what activities are intentionally excluded from this activity (Among other things, if
the activity is a service like e.g. spinning of bast fibres, that does not include the product used in
the process (i.e. the bast fibres), this information will be included here).  

Each part has a specific mandatory wording. I suggest providing default text for the three sections separately and allowing the user to modify as required.  
Suggested text:

In [59]:
includedActivitiesEndText_last = "This activity ends with the discharge of treated wastewater to the natural environment\
(treated fraction) and with the discharge of untreated wastewater to the natural environment (untreated fraction)."

includedActivitiesEndText_included =\
"This activity includes the transportation of wastewater via the sewer grid,\
the treatment of the wastewater in the wastewater treatment plant as well as the fraction of\
wastewater discharged to the sewer grid but ultimately not treated (sewer unconnected to wastewater treatment\
plant or discharge due to hydraulic overload). The amounts of infrastructure and consumables are\
also included as inputs to the activity."

includedActivitiesEndText_excluded = \
" By definition, wastewater not sent to the sewer grid is also excluded. The production of sludge is included, but its \
treatment is covered by another treatment activity."

In [60]:
"{} {} {}".format(
    includedActivitiesEndText_last,
    includedActivitiesEndText_included,
    includedActivitiesEndText_excluded
)

'This activity ends with the discharge of treated wastewater to the natural environment(treated fraction) and with the discharge of untreated wastewater to the natural environment (untreated fraction). This activity includes the transportation of wastewater via the sewer grid,the treatment of the wastewater in the wastewater treatment plant as well as the fraction ofwastewater discharged to the sewer grid but ultimately not treated (sewer unconnected to wastewater treatmentplant or discharge due to hydraulic overload). The amounts of infrastructure and consumables arealso included as inputs to the activity.  By definition, wastewater not sent to the sewer grid is also excluded. The production of sludge is included, but its treatment is covered by another treatment activity.'

##### Adding the includedActivitiesStart, includedActivitiesEnd

In [27]:
generate_activity_boundary_text(dataset,
                               includedActivitiesStartText,
                               includedActivitiesEndText_last,
                               includedActivitiesEndText_included,
                               includedActivitiesEndText_excluded)

#### Technology level

From the Data Quality Guidelines:

>The technology level of a transforming activity is classified in one of these five classes:  
0=Undefined. For market activities that do not have a technology level.  
1=New. For a technology assumed to be on some aspects technically superior to modern technology, but not yet the most commonly installed when investment is based on purely economic considerations.  
2=Modern. For a technology currently used when installing new capacity, when investment is based on purely economic considerations (most competitive technology).  
3=Current (default). For a technology in between modern and old.  
4=Old. For a technology that is currently taken out of use, when decommissioning is based on purely economic considerations (least competitive technology).  
5=Outdated. For a technology no longer in use.The terms used does not necessarily reflect the age of the technologies.   
A modern technology can be a century old, if it is still the most competitive technology, and an old technology can be relatively young, if it is one that has quickly become superseded by other more competitive ones. The technology level is relative to the year for which the data are valid, as given under Time Period. In a time series, the same technology can move between different technology levels over time. The same technology can also be given different technology levels in different geographical locations, even in the same year.  

The user should be able to choose the most relevant technology level from a drop-down menu, have access to descriptions and have the tool default to an appropriate value (current).

In [28]:
generate_technology_level(dataset, 'Current')

#### Activity-level comment fields

This is a list of text cells. (Images are also legal, but I'm not sure how we could handle that, nor if they are useful)  

We should supply default text and allow the users to change the text if necessary.  

We should also determine how many cells we want to provide (different cells are listed one after another in ecoEditor, and this seperation is used to split different subjects).

I have written a generic function to add comment fields, `generate_comment` that takes the following arguments:  
- the `dataset`  
- the `comment_type`. Valid types are 'allocationComment','generalComment','geographyComment','technologyComment' and 'timePeriodComment'.  
- the list of text comments (one per cell). 

##### Technology description
>Text (_and image, but I'm not sure how we'd handle that_) field to describe the technology of the activity. The text should cover information necessary to identify the properties and particularities of the technology(ies) underlying the activity data. Describes the technological properties of the unit process. If the activity comprises several subactivities, the corresponding technologies should be reported as well. Professional nomenclature should be used for the description.  

*We should discuss how default text can be generated here based on the modelled technology.* Here's assuming three cells.

In [29]:
tech_comment_1 = 'The technologies modelled are x and y'
tech_comment_2 = 'They were averaged based on z'
tech_comment_3 = 'These technologies rock'

generate_comment(dataset, 'technologyComment', [tech_comment_1, tech_comment_2, tech_comment_3])

##### generalComment
From the [dataset documentation](http://www.ecoinvent.org/files/dataset_documentation_ecoinvent_3.pdf) document:  
>Information that concerns the construction of the inventory (details about the Functional Unit, background,
etc.) shall be entered in the General Comment field. Actually, this field can be compared to
the abstract of a scientific article – i.e. the field should offer to the user a first, rough overview of the
dataset.
Please start the text always with "This dataset represents [the production]/ [the service of] ...."  

We should provide default text and allow the user to modify. Again, multiple cells are possible. 

In [61]:
general_comment_1 = 'This dataset represents the treatment of wastewater discharged to the sewer grid from {}'.format(
    'the production of ceramic')
general_comment_2 = 'It includes the transportation of the wastewater to the wastewater treatment plant and the actual treatment,\
 as well as wastewater discharged from the sewer grid to the natural environment without treatment.'
general_comment_3 = 'It was modelled using XYZ'

generate_comment(dataset, 'generalComment', [general_comment_1, general_comment_2, general_comment_3])

#####  'geographyComment', 'timePeriodComment'
The user _could_ want to include a comment. 
>**'timePeriodComment'** Text and image field for additional explanations concerning the temporal validity of the flow data reported. It may e.g. include information about:
- how strong the temporal correlation is for the unit process at issue (e.g., are four year old data still adequate for the activity operated today?)  

> **'geographyComment'**  Especially for area descriptions, the nature of the geographical delimitation may be given, especially when this is not an administrative area.  

Let's suppose no such comment now.

In [31]:
generate_comment(dataset, 'timePeriodComment', ['This is a time period comment'])
generate_comment(dataset, 'geographyComment', ['This is a geography comment'])

##### 'allocationComment'
I would leave this out - I don't see the user needing this.

### 'modellingAndValidation', Representativeness
There are multiple fields that are filled in by default (by function). The tool/user should provide, insofar as possible information on three specific items:  
 - 'samplingProcedure'  
>Text describing the sampling and calculation procedures applied for quantifying the exchanges. Reports whether the sampling procedure for particular elementary and intermediate exchanges differ from the general procedure. Mentions possible problems in combining different sampling procedures.  

I will let ICRA generate default text for this field.

 - 'Extrapolations'
 
> Describes extrapolations of data from another time period, another geographical area or another technology and the way these extrapolations have been carried out. It should be reported whether different extrapolations have been done on the level of individual exchanges. If data representative for a activity operated in one country is used for another country's activity, its original representativity can be indicated here. Changes in mean values due to extrapolations may also be reported here.

We should talk about the text text to include here.

`percent`
> Percent of data sampled out of the total that the activity is intended to represent (as given by the fields geography, technology and time period).

Perhaps blank as default (allowed), with the option to add information if available. 

Putting it all together in a function `generate_representativeness(dataset, samplingProcedure_text, extrapolations_text, percent)`

In [32]:
samplingProcedure_text = 'This is a description of the sampling procedure, and it should be changed by IRCA'
extrapolation_text = 'This is a description of the sampling procedure, and it should be changed by IRCA'
percent = 80

generate_representativeness(dataset, samplingProcedure_text, extrapolation_text, percent)

### Data entry section:  
For now, filled in with dummy data ([Current User]). Gets populated by ecoEditor.  I moved the whole section to `create_empty_dataset`

### DataGeneratorAndPublication
The data here should basically be the reference of the tool. We will need to include the following information:
  - Author (called dataGenerator)
  - PublishedSource (reference, if there is a report or paper coming out of this work). If we don't publish, this is empty and the `dataPublishedIn` is set to 0.  
  - pageNumber  
  - ... see ecoEditor. 
For now, I'll assume no publication and put myself as author (this will need to change).

The users will have the possibility to change all this in the ecoEditor, which is probably easier than doing it in the tool.

## Exchanges!!

### Reference exchange (reference flow)
This is the amount of wastewater treated. 
Here are some key things to know about this exchange:  
  - It needs to be expressed in m3  
  - Its amount is -1 (1 because it is the common denominator for the whole dataset, and - because this is a convention in ecoinvent to identify treated exchanges)  
  - The name is auto-generated by the `generate_reference_exchange` function  
  - **Important (easy place to introduce a mistake)** The reference flow refers to the WW sent ot the sewer system, and not for the WW actually treated in the WWTP. In other words, it accounts for both the overflow fraction and the treated fraction. 
  - It needs to be associated with some properties (BOD5, metal content, etc). For now, I have stored these in an Excel file. How the tool collects this information and passes it to the function below that generates the dataset will be up to ICRA).  
  - The uncertainty of the reference exchange is nul (it is the only exchange without uncertainty). 
  - The reference exchange shall also be accompanied by a comment. I propose a default comment below.  
  - A production volume needs to be defined. It is equal to the total amount of hte WW of interest sent to the sewer in the regional scope of the dataset. It is expressed in m3/year. For "average" WW, we can possibly find default numbers. For industrial WW, we should provide guidance on how to generate this value (total production volume of the production dataset * the amount of WW generated per unit produced)  
  - The production volume needs to accompanied with a comment. We could determine what default comment would be appropriate once we determine how the default value for the production volume will be calculated.  
  - The production volume also needs to be accompanied with an uncertainty dictionary. The uncertainty dictionary has the following format: `{'variance':basic uncertainty with a default value of 0.0006, 'pedigreeMatrix':[scores from 1-5 for five indicators]}`. The default pedigree scores will depend on how we generate the default production volume amount. The scores refer to the following (to be included in the tool for all exchanges):
  ![Pedigree scores](pix\pedigree_scores.PNG)

#### Obligatory properties for WW
For now, I am storing values in Excel, and generating a pandas DataFrame. 

In [33]:
def get_WW_properties(xls=None):
    # From excel for now
    if xls==None:
        xls = os.path.join(root_dir, 'Documentation', 'WW_properties.xlsx')
    return pandas.read_excel(xls, sheet_name='Sheet1', index_col=1)

In [34]:
WW_prop_df = get_WW_properties()

In [35]:
WW_prop_df.head()

,id,unitName,comment,Amount,Variance,pedigree1,pedigree2,pedigree3,pedigree4,pedigree5,2.2_cat,2.2_cat_comment,2.2_comment,2.2_name
name,,,,,,,,,,,,,,
"BOD5, mass per volume",dd13a45c-ddd8-414d-821f-dfe31c7d2868,kg/m3,"Biological Oxygen Demand BOD5, as O2",0.336730,0.084773,2,4,3,5,4,Carbon,all combinations are possible,NaN,Chemical Oxygen Demand COD as O2
"COD, mass per volume",3f469e9e-267a-4100-9f43-4297441dc726,kg/m3,Chemical Oxygen Demand as O2,0.242895,0.518737,4,4,1,5,4,C,NaN,BOD (preferred),Biological Oxygen Demand BOD5 as O2
"mass concentration, DOC",efe22a60-b1a3-4b33-a5ba-4bf575e0a889,kg/m3,Mass concentration of Dissolved Organic Carbon,0.151075,0.599728,2,5,5,3,4,C,NaN,NaN,Dissolved organic carbon DOC as C
"mass concentration, TOC",a547f885-601d-4d52-9bf9-60f0cef06269,kg/m3,Mass concentration of Total Organic Carbon,0.416505,0.094506,3,2,2,4,5,C,NaN,NaN,Total organic carbon TOC as C
"mass concentration, dissolved sulfate SO4 as S",1e4ef691-c7d3-49fc-9aee-6d77575a7b8a,kg/m3,Mass concentration of dissolved sulfate SO4 (C...,0.770858,0.261701,5,2,3,1,1,Sulfur,either,SO4 & S-part.,Sulfate SO4 as S


The DataFrame `WW_prop_df` then gets converted to a list of tuples that are used to generate the ecoSpold2 file.  

**Important**: For now, it is assumed that the properties are not given a "variable name", which would allow them to easily be used in equations within ecoEditor. If we decide to use equations, we should add this field to the tuple. There are **numerous** advantages to doing this (two most important: transparency, and propagation of uncertainty). 

In [36]:
def convert_WW_prop_to_list(df):
    #(property_name, amount, unit, comment, uncertainty)
    return [(i,
             df.loc[i, 'Amount'],
             df.loc[i, 'comment'],
             df.loc[i, 'unitName'],
             {
                 'variance': df.loc[i, 'Variance'],
                 'pedigreeMatrix': [
                     df.loc[i, 'pedigree1'],
                     df.loc[i, 'pedigree2'],
                     df.loc[i, 'pedigree3'],
                     df.loc[i, 'pedigree4'],
                     df.loc[i, 'pedigree5'],
                 ]
             }
            ) for i in df.index]

In [37]:
convert_WW_prop_to_list(WW_prop_df)

[('BOD5, mass per volume',
  0.3367304481602893,
  'Biological Oxygen Demand BOD5, as O2',
  'kg/m3',
  {'pedigreeMatrix': [2, 4, 3, 5, 4], 'variance': 0.084772935248650816}),
 ('COD, mass per volume',
  0.24289533509543038,
  'Chemical Oxygen Demand as O2',
  'kg/m3',
  {'pedigreeMatrix': [4, 4, 1, 5, 4], 'variance': 0.51873718055969376}),
 ('mass concentration, DOC',
  0.15107453144940808,
  'Mass concentration of Dissolved Organic Carbon',
  'kg/m3',
  {'pedigreeMatrix': [2, 5, 5, 3, 4], 'variance': 0.59972841525803788}),
 ('mass concentration, TOC',
  0.41650470078041757,
  'Mass concentration of Total Organic Carbon',
  'kg/m3',
  {'pedigreeMatrix': [3, 2, 2, 4, 5], 'variance': 0.094505715185841388}),
 ('mass concentration, dissolved sulfate SO4 as S',
  0.77085844381587831,
  'Mass concentration of dissolved sulfate SO4 (CAS 014808-79-8) as S. Note that dissolved and particulate S are optional yet desirable specifications of total sulfur.',
  'kg/m3',
  {'pedigreeMatrix': [5, 2, 

#### Other inputs required for the reference exchange

It is assumed the tool will generate these. Code for the default comments could be reused:

In [38]:
WW_discharged_without_treatment = 0.1 # Fraction of water sent to sewer grid but not treated in WWTP (e.g. overflow)

base_ref_exchange_comment = "Refers to the amount of wastewater sent to the sewer grid for treatment."
if WW_discharged_without_treatment != 0:
    ref_exchange_comment = base_ref_exchange_comment +\
                            " Includes fraction ({:.0f}%) of wastewater sent to sewer grid but not treated in a wastewater treatment plant (e.g. overflow).".format(
                                WW_discharged_without_treatment*100)
else:
    ref_exchange_comment = base_ref_exchange_comment

PV = 1000000 # Some fake number, we need to determine how to calculate this.

base_PV_comment = "Yearly volume of wastewater, {}, sent to the sewer grid for treatment".format(dataset['WW_type'],)
if WW_discharged_without_treatment != 0:
    PV_comment = base_PV_comment +\
                "Includes the fraction that is treated ({:.0f}%) and the fraction that is discharged directly to the environment as overflow ({:.0f}%).".format(
                    (1-WW_discharged_without_treatment)*100,
                    WW_discharged_without_treatment*100
                )
else:
    ref_exchange_comment = base_ref_exchange_comment

PV_uncertainty = {'variance':0.01, 'pedigreeMatrix':[2,4,3,2,4]} # Fake numbers for now.  

In [39]:
ref_exchange_comment

'Refers to the amount of wastewater sent to the sewer system for treatment. Includes fraction (10%) of wastewater sent to sewer system but not treated in a wastewater treatment plant (e.g. overflow).'

In [40]:
PV_comment

'Yearly volume of wastewater, from ceramic production, sent to the sewer system for treatmentIncludes the fraction that is treated (90%) and the fraction that is discharged directly to the environment as overflow (10%).'

#### Using the function to generate the reference exchange

In [41]:
dataset, MD = generate_reference_exchange(dataset, 
                          exc_comment=ref_exchange_comment,
                          PV=PV,
                          PV_comment=PV_comment,
                          PV_uncertainty=PV_uncertainty,
                          MD=MD)

### Byproducts/wastes  
There are two types of wastes/byproducts to be considered:  
1) Sludge  
2) Grit  

#### Sludge
Sludge treatment is outside the scope of this project. 
However, the tool must generate a "sludge exchange" that indicates how much sludge is generated (per reference exchange) and what its composition is (via its "properties").  
The properties of the sludge will be dictated by the transfer coefficients the tool will calculate.   
There are three approaches here: 
- Enter the transfer coefficients in the dataset, and let the sludge composition be calcualted within the ecoSpold file itself, by ecoinvent (better for transparency and uncertainty propagation)  
- Calculate the sludge properties and enter them as static values, but include the transfer coefficients in the comments (good for transparency).  
- Calculate the sludge properties and enter them as static values with generic comments (worst for transparency)  

I'll assume for now that option 2 is chosen. 

##### Transfer coefficients  
These will be calculated, but I will store them in a table in the excel spreadsheet "sludge_transfer_coeff.xlsx" for now.  

In [42]:
r'C:\mypy\code\wastewater_treatment_tool\waste_water_tool\documentation\sludge_transfer_coeff.xlsx'

'C:\\mypy\\code\\wastewater_treatment_tool\\waste_water_tool\\documentation\\sludge_transfer_coeff.xlsx'

In [43]:
def get_WW_properties(xls=None):
    # From excel for now
    if xls==None:
        xls = os.path.join(root_dir, 'Documentation', 'sludge_transfer_coeff.xlsx')
    return pandas.read_excel(xls, sheet_name='Sheet1', index_col=1, )

**REST DEPENDS ON ADDING PROPERTIES ON THE FLY**

##### Grit
I will assume here, like in ecoinvent, that there are two types of grit: 
  - plastics  
  - biomass, modelled as paper  
  
The tool should provide default values for amounts of grit removed, as well as the uncertainty for these.  
The default values in ecoinvent v2.2 are 15.5 g/m3 of each, and the basic uncertainty is 0.0006.  
If we use these values, the pedigree scores should be: [1,3,5,5,1]  
It would be MUCH better to use other data for this.

In [44]:
grit_default_total_amount = 0.031 #kg/m3 in WWTP - ideally this value would be updated, and in any case the user can override it
grit_default_plastic_ratio = 0.5
grit_default_biomass_ratio = 0.5
grit_uncertainty = {'variance':0.0006, 'pedigreeMatrix':[1,3,5,5,1]} #If we use the ecoinvent v2.2 data.
grit_plastics_comment_default = "Amount of plastics removed from wastewater. Based on an assumed {} kg/m3 of grit removed, \
an assumed {:2}% of the grit that is plastics, and an assumed {:2}% of discharged \
wastewater not treated at the wastewater treatment plant".format(grit_default_total_amount,
                                                                 grit_default_plastic_ratio*100,
                                                                 WW_discharged_without_treatment*100)
grit_biomass_comment_default = "Amount of biomass  removed from wastewater. Based on an assumed {} kg/m3 of grit removed, \
an assumed {:2}% of the grit that is biomass, and an assumed {:2}% of discharged \
wastewater not treated at the wastewater treatment plant. Biomass waste management modelled as paper waste management".format(
    grit_default_total_amount,
    grit_default_biomass_ratio*100,
    WW_discharged_without_treatment*100)

In [45]:
grit_plastics_comment_default

'Amount of plastics removed from wastewater. Based on an assumed 0.031 kg/m3 of grit removed, an assumed 50.0% of the grit that is plastics, and an assumed 10.0% of discharged wastewater not treated at the wastewater treatment plant'

In [46]:
grit_biomass_comment_default

'Amount of biomass  removed from wastewater. Based on an assumed 0.031 kg/m3 of grit removed, an assumed 50.0% of the grit that is biomass, and an assumed 10.0% of discharged wastewater not treated at the wastewater treatment plant. Biomass waste management modelled as paper waste management'

In [48]:
dataset =  add_grit(dataset,
             grit_default_total_amount,
             WW_discharged_without_treatment,
             grit_default_plastic_ratio,
             grit_default_biomass_ratio,
             grit_uncertainty,
             grit_plastics_comment_default,
             grit_biomass_comment_default,
             PV,
             MD)

### Inputs from the technosphere  
Inputs from the technosphere correspond to consummables, energy and infrastructure inputs. Here is the list in the current WWT datasets:  
  - Consumables:  
    - iron (III) chloride, without water, in 40% solution state  
    
- Energy:  
    - heat, district or industrial, natural gas  
    - heat, central or small-scale, other than natural gas  
    - electricity, low voltage  
    
    
- Infrastructure:  
  - wastewater treatment facility, capacity XXl/year  
  - sewer grid, XXl/year, YY km    

#### Consumables:
  - aluminium sulfate, powder  
  - aluminium sulfate, without water, in 4.33% aluminium solution state  
  - lime  
  - iron(III) chloride, without water, in 14% iron solution state  
  
Required inputs: amount only.

##### Sewer grids: 
- There are currently five sewer grid construction/repair/end-of-life datasets in ecoinvent, each associated with different WWTP capacities (smaller grids have smaller diameters, are longer per capita and transport less WW over their lifetime).  
- In order to know how to model these, we need to decide whether:  
  - We want to create new datasets for this (I suppose not).  
  - We want to 

### Elementary flows - Direct discharge  
Direct emissions associated with the share of wastewater discharged to the sewer grid but not treated (unconnected sewers or hydraulic overload). Chemical reactions of the wastewater within the sewer are not considered.   

# TBC

In [49]:
MD['Properties'].columns

Index(['comment', 'id', 'unitName'], dtype='object')

In [50]:
generate_ecoSpold2(dataset,
                   r'C:\mypy\code\wastewater_treatment_tool\waste_water_tool\templates',
                   'test_1232.spold',
                   r'C:\mypy\code\wastewater_treatment_tool\waste_water_tool\result_folder')

Dataset_2.xml
TTextAndImage_2.xml
TTextAndImage_2.xml
TClassification_2.xml
DataEntryBy_2.xml
ModellingAndValidation_2.xml
TTextAndImage_2.xml
Exchange_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TUncertainty_2.xml
TProperty_2.xml
TU